XML stands for the eXtensible Markup Language.  It appeared as a successor to SGML (standard generalized markup language) and HTML (hypertext markup language, the standard for displaying web pages), but with some additional structure that makes the documents more well-defined; for instance, in HTML it's common for open tags to appear without a corresponding close tag, which is not allowed in pure XML.

You may already be familiar with XML, but if not the official resource for learning about the format is here https://www.w3.org/XML/ and a good resource with some concrete examples is here http://www.w3schools.com/xml/.  We'll assume here that you're broadly familiar with the basic ideas behind XML, and just describe what you need to know to complete the parser for this assignment.

Here is an example XML document:
    
    <?xml version="1.0" encoding="UTF-8"?>
    <!-- This is a comment -->
    <note date="8/31/12">
        <to>Tove</to>
        <from>Jani</from>
        <heading type="Reminder"/>
        <body>Don't forget me this weekend!</body>
        <!-- This is a multiline comment,
             which take a bit of care to parse -->
    </note>
    
There are a few elements here of importance.
1. Tags are denoted `<tag_name>content</tag_name>` where `<tag_name>` is the opening tag and `</tag_name>` is the closing tag.  All text (including whitespace, and subtags, etc) between these tags is the content.
2. Attributes follow a tag, and are written as a list of `attr_name="attribute_value"` pairs, where we can use either double quotes or single quotes around the attribute value.  If you use double quotes then a single quote can appear in the text and vice versa.  There can be whitespace around the equals sign or not.
3. If a tag has no content `<tag_name attr_name="attribute_value"></tag_name>` can be abbreviated as the open/close tag `<tag_name attr_name="attribute_value"/>`. In some cases, such as in HTML5, you might come across tags that have no content but aren't closed, such as `<meta ...>` and `<link ...>` tags. **However, as we are dealing with XML (a stricter context), tags are required to be closed.** If you're interested, [this document](https://www.w3schools.com/html/html_xhtml.asp) on XHTML vs HTML touches upon this idea.
4. A XML prologue is written as `<?tag_name attr_name="attribute_value"?>`.  It has no close tag.  We'll also consider documents that allow for an HTML declaration, such as `<!DOCTYPE html>` (this will let us parse some HTML documents that are well-formed enough to also parse as valid XML).
5. Comments are denoted by `<!-- comment_text -->`  and the comment text can span multiple lines.

## Q1: Regular expression for identifying tags

First, we'll use regular expressions to identify tags and other elements of XML files.  Specifically, you'll need to create 6 regular expressions that locate open tags, close tags, open/close tags, comments, xml_prolog, and html declarations.  For the open, close and open/close tags, make sure that your regular expression also matches and returns 1) the tag name, and 2) all the attributes (i.e., they should contain two groups, one for each of these two components).

Note that in order to match the solutions, your `tag_open` regex should _both_ open _and_ open/close tags (it's posssible, but substantially trickier, to create regex that matches _only_ open and _not open/close tags, so we'll handle that logic separate).  Comments may be split across multiple lines, but you can assume that all other tags must occur on a single line (without newlines within the tag itself).

Fill in the in the following regular expressions.

# XML Parser


In this problem, you're going to write a function that will parse (a simplified version of) XML files into a Python object. Although it's not recommended that you use the parser you construct for anything serious (many excellent Python libraries already exist for parsing XML, such as the lxml library), XML files represent a fairly complex file format, that necessitates using regular expressions and recursion (or a stack) to parse these in a reasonably efficient manner.  So while it's not likely that you will need to write you own XML parser, chances are if/when you _do_ need to write a parser for some format for which there exists no good Python library, the techniques you use here will be useful for writing this parser as well.

In [1]:
import re
import requests

tag_open = re.compile("")
tag_close = re.compile("")
tag_open_close = re.compile("")

comment = re.compile("")
xml_prolog = re.compile("")
html_declaration = re.compile("")

def tag_regex(inp):
    return {
        "tag_open": tag_open.findall(inp),
        "tag_close": tag_close.findall(inp),
        "tag_open_close": tag_open_close.findall(inp),
        "comment": comment.findall(inp),
        "xml_prolog": xml_prolog.findall(inp),
        "html_declaration": html_declaration.findall(inp)
    }

## Q2: XML Parser

Using the regular expressions above, now you'll write an XML parser (although technically you don't _have_ to use them, you could try to write a complete XML parser using a single regular extended expression if you really want to, but we would highly advise against this).  Specifically, you should fill in the `__init__` function for the class prototype below (more discussion of the actual implemeentation follows below).

We've provided you with some test cases to make sure that your parser works correctly.

In [2]:
import re

tag_open = re.compile(r"<(\w+)([^>]*)>(.*?)</\1>|<(\w+)([^>]*)/>")
tag_close = re.compile(r"</(\w+)[^>]*>")
comment = re.compile(r"<!--(.*?)-->", re.DOTALL)
xml_prolog = re.compile(r"<\?([\w-]+)(.*?)\?>")
html_declaration = re.compile(r"<!DOCTYPE\s+\w+\s*\[.*?\]>\s*|<!DOCTYPE\s+\w+>")
class XMLNode:
    def __init__(self, tag, attributes, content):
        self.tag = tag # The tag <tag>
        self.attributes = attributes # A dictionary from attributes to values
        self.children = [] # A list of either XMLNode objects and strings, corresponding to tags and text between them.
        self.content = content # A string of everything in the original document inside these tags

    def create_xml_tree(text):
        """parse an XML tree from a string"""
        pos = 0

        def parse_tags(inp):
            nonlocal pos
            nodes = []

            while pos < len(inp):
                match = tag_open.search(inp, pos)
                if match:
                    if match.start() > pos:
                        nodes.append(inp[pos:match.start()])

                    tag_name = match.group(1)
                    attributes = dict(re.findall(r'\b(\S+?)\s*=\s*(?:"([^"]*)"|\'([^\']*)\')', match.group(2)))
                    if tag_close.search(inp, match.end()):
                        end_match = tag_close.search(inp, match.end())
                        nodes.append(XMLNode(tag_name, attributes, inp[match.end():end_match.start()]))
                        pos = end_match.end()
                        return nodes
                    elif tag_open_close.search(inp, match.end()):
                        end_match = tag_open_close.search(inp, match.end())
                        nodes.append(XMLNode(tag_name, attributes, ''))
                        pos = end_match.end()
                    else:
                        raise ValueError("Invalid XML format")
                else:
                    nodes.append(inp[pos:])
                    pos = len(inp)
            return nodes

        root = XMLNode("", {}, "")
        stack = [root]
        nodes = parse_tags(text)

        for node in nodes:
            if isinstance(node, str):
                stack[-1].children.append(node)
            else:
                stack[-1].children.append(node)
                if node.content:
                    stack.append(node)
                else:
                    stack[-2].children.append(node)

        return root

                # Fill out this function and move it into the XMLNode class above.
    def find(self, tag, **kwargs):
        """
        Search for a given tag and atributes anywhere in the XML tree

        Args:
            tag (string): tag to match
            kwargs (dictionary): list of attribute name / attribute value pairs to match

        Returns:
            (list): a list of XMLNode objects that match from anywhere in the tree
        """
        found_nodes = []

          # Check if the current node matches the search criteria
        def node_matches_criteria(node):
            if node.tag != tag:
                return False

            for attr, value in kwargs.items():
                if node.attributes.get(attr) != value:
                    return False

            return True

        # Recursive function to search for nodes
        def search_nodes(current_node):
            if node_matches_criteria(current_node):
                found_nodes.append(current_node)

            for child in current_node.children:
                if isinstance(child, XMLNode):
                    search_nodes(child)

        # Start searching from the root node (self)
        search_nodes(self)
        return found_nodes

    # Test the XML parser
xml_string = """
<note date="8/31/12">
    <to>Tove</to>
    <from>Jani</from>
    <heading type="Reminder"/>
    <body>Don't forget me this weekend!</body>
</note>
"""
xml_tree = XMLNode.create_xml_tree(xml_string)

In [3]:
def print_xml_tree(node, depth=0):
    indent = "  " * depth
    if node.tag:
        print(f"{indent}Tag: {node.tag}, Attributes: {node.attributes}")
    if node.content:
        print(f"{indent}Content: {node.content.strip()}")

    for child in node.children:
        if isinstance(child, XMLNode):
            print_xml_tree(child, depth + 1)
        else:
            print(f"{indent}Text: {child.strip()}")

# Assuming 'xml_tree' is the root node obtained from parsing the XML string
print_xml_tree(xml_tree)


Text: <note date="8/31/12">
  Tag: to, Attributes: {}
  Content: <from>Jani


Lets discuss in a bit more detail how the XML parsing will work algorithmically.  We begin the initializer by copying the provided parameters to the class attributes.  Note that if you want you could make a full string copy here, but we don't bother.  Now we begin parsing the file, which we do by repeating the following logic until termination:
1. Look for the next xml tag (or comment, etc), in the file.  This is best done by finding the next `'<'` character.  If you can't find any, return.
2. If it's an xml prolog, html declaration, or comment, ignore this portion, and continue parsing after the prolog, declaration, or comment (i.e., throw away whatever information is contained in these portions)
3. If it's an open tag, read its tag and attributes (you'll likely want to use a regular expression to parse the attributes as well, but we leave this up to you).  If it's just an open tag, then recursively create an XMLNode object initializer this tag and attributes, and the content that occurred after the open tag.  If it's an open/close tag, create a XMLNode the same as before but with empty content.
4. If it's a close tag, make sure that the close tag matches the tag originally provided to the current XMLNode constructor (otherwise, we have a situation where one tag is closed with a different tag), and raise an Exception if not.  If the tags do match, then truncate the content to contain only the content before the closed tag matched, and return.

Some hints that we believe will be helpful:
1. Keep track of the current position where you are parsing the file, and make sure to properly increment this so you move past any tag that you have parsed.
2. Make use of the `match = regular_expression_obj.match(string, pos)` function, which looks for a match to the regular expression starting _exactly_ as position `pos` in `string`.  If this function returns `None`, then the regular expression did not match.  In the returned `match` object, `match.end()` contains the position where the match ended.

## Q3: Searching for tags

One of the nicer elements of the `BeautifulSoup` library is the ability to quickly search for tags that have certain attributes, without worrying about the specific structure of the model (i.e., how many levels deep the tag is, how many may exist in the document etc).  We're going to implement a similar function in our `XMLNode` class

For those who haven't seen the `**kwargs` parameter before, this is just a way to pass a variable-length list of parameters to a Python function as function parameters.  For example, you could call `find` via

```python
root.find("link", rel="stylesheet") -> [XMLNode("link", {...}, "...")]
```

and in the `find` function, `kwargs` would be a dictionary equal to `{"rel":"stylesheet"}`.

This function should return a list of _all_ XMLNodes that are descendents (children, children of children, etc), of the node you call it on. Providing no filter should match all nodes.


In [4]:
# Fill out this function and move it into the XMLNode class above.
def find(self, tag, **kwargs):
    """
    Search for a given tag and atributes anywhere in the XML tree

    Args:
        tag (string): tag to match
        kwargs (dictionary): list of attribute name / attribute value pairs to match

    Returns:
        (list): a list of XMLNode objects that match from anywhere in the tree
    """
    found_nodes = []

      # Check if the current node matches the search criteria
    def node_matches_criteria(node):
        if node.tag != tag:
            return False

        for attr, value in kwargs.items():
            if node.attributes.get(attr) != value:
                return False

        return True

    # Recursive function to search for nodes
    def search_nodes(current_node):
        if node_matches_criteria(current_node):
            found_nodes.append(current_node)

        for child in current_node.children:
            if isinstance(child, XMLNode):
                search_nodes(child)

    # Start searching from the root node (self)
    search_nodes(self)
    return found_nodes

After you have created the class with this `find` function, you can run the following tests.

In [5]:
def create_searchable_xml_tree(text):
    """parse an XML tree from a string"""
    #return XMLNode("", {}, text)
    pos = 0

    def parse_tags(inp):
        nonlocal pos
        nodes = []

        while pos < len(inp):
            match = tag_open.search(inp, pos)
            if match:
                if match.start() > pos:
                    nodes.append(inp[pos:match.start()])

                tag_name = match.group(1)
                attributes = dict(re.findall(r'\b(\S+?)\s*=\s*(?:"([^"]*)"|\'([^\']*)\')', match.group(2)))
                if tag_close.search(inp, match.end()):
                    end_match = tag_close.search(inp, match.end())
                    nodes.append(XMLNode(tag_name, attributes, inp[match.end():end_match.start()]))
                    pos = end_match.end()
                    return nodes
                elif tag_open_close.search(inp, match.end()):
                    end_match = tag_open_close.search(inp, match.end())
                    nodes.append(XMLNode(tag_name, attributes, ''))
                    pos = end_match.end()
                else:
                    raise ValueError("Invalid XML format")
            else:
                nodes.append(inp[pos:])
                pos = len(inp)
        return nodes

    root = XMLNode("", {}, "")
    stack = [root]
    nodes = parse_tags(text)

    for node in nodes:
        if isinstance(node, str):
            stack[-1].children.append(node)
        else:
            stack[-1].children.append(node)
            if node.content:
                stack.append(node)
            else:
                stack[-2].children.append(node)

    return root

# Assuming 'xml_tree' is the root node obtained from parsing the XML string
xml_string = """
<note date="8/31/12">
    <to>Tove</to>
    <from>Jani</from>
    <heading type="Reminder"/>
    <body>Don't forget me this weekend!</body>
</note>
"""
xml_tree = create_searchable_xml_tree(xml_string)

In [6]:
# Test the XML parser
found_nodes = xml_tree.find("body")
for node in found_nodes:
    print(f"Tag: {node.tag}, Attributes: {node.attributes}, Content: {node.content.strip()}")

In [7]:
import random

# A set containing elements of different datatype
set_my = {'rock', 'paper', 'scissors'}

print("An element from the set:" , random.sample(set_my, 1))

An element from the set: ['paper']


<ipython-input-7-c2c716f2fce6>:6: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  print("An element from the set:" , random.sample(set_my, 1))
